In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import copy
import pandas as pd
from tqdm import tqdm
from typing import Counter
import math
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.svm import SVC


def load_data():
    data = loadmat("data/mnist_all.mat")

    # print(data.keys())

    train_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for i in range(10):
        temp_df = pd.DataFrame(data["train" + str(i)])
        temp_df['label'] = i
        train_data = train_data.append(temp_df)
        temp_df = pd.DataFrame(data["test" + str(i)])
        temp_df['label'] = i
        test_data = test_data.append(temp_df)

    train_data = shuffle(train_data)
    test_data = shuffle(test_data)

    train_labels = np.array(train_data['label'])
    test_labels = np.array(test_data['label'])

    train_data = train_data.drop('label', axis=1)
    test_data = test_data.drop('label', axis=1)
    
    train_data = np.array(train_data) / 255
    test_data = np.array(test_data) / 255
    
    pca = PCA(0.95)
    pca.fit(train_data)
    train_data = pca.transform(train_data)
    test_data = pca.transform(test_data)

    return train_data, test_data, train_labels, test_labels


X_train, X_test, y_train, y_test = load_data()

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0)

In [2]:
print(X_train.shape)

(48000, 154)


In [3]:
class SVM:
    def __init__(self, C):
        self.C = C
        self.b = 0
    
    def train(self, X_data, Y_data, valid_x, valid_y, epochs, kernel_type):
        self.x_num = X_data.shape[0]
        
        self.epochs = epochs
        self.X_data = X_data
        self.Y_data = Y_data
        
        self.valid_x  = valid_x
        self.valid_y = valid_y
        
        self.E = np.zeros(self.x_num)
        self.alpha = np.zeros(self.x_num)
        
        self.best_alpha = None
        self.best_f1 = 0
        self.best_b = None
        
        self.kernel_type = kernel_type
        
        self.kernel_matrix = np.zeros(shape=(self.x_num, self.x_num))
        
        self.valid_kernel_matrix = np.zeros(shape=(self.x_num, len(self.valid_y)))
        
        self.init_valid_kernel_matrix()
        self.init_kernel_matrix()
        self.init_E()
        
        pair_changed = 0
        for epoch in range(self.epochs):
            for i in range(self.x_num):
    
                if not ((self.Y_data[i] * self.E[i] < -0.001) and (self.alpha[i] <self.C) or \
                     (self.Y_data[i] * self.E[i] > 0.001) and (self.alpha[i] > 0)):
                    continue
                
                val, j = self.select_j(i)
                
                while j < self.x_num:
                    if i == j:
                        j += 1
                        continue
                    if self.Y_data[i] == self.Y_data[j]:
                        L = max(0, self.alpha[j] + self.alpha[i] - self.C)
                        H = min(self.C, self.alpha[j] + self.alpha[i])
                    else:
                        L = max(0, self.alpha[j] - self.alpha[i])
                        H = min(self.C, self.C + self.alpha[j] - self.alpha[i])

                    if L == H:
                        j += 1
                        continue
                    old_alpha_j = self.alpha[j]
                    yita = self.kernel_matrix[i][i] + self.kernel_matrix[j][j] \
                               - 2 * self.kernel_matrix[i][j]

                    if yita <= 0:
                        j += 1
                        continue

                    new_alpha_unc_j = old_alpha_j + self.Y_data[j] * (self.E[i] - self.E[j]) / yita

                    if new_alpha_unc_j >= H:
                        new_alpha_clippd_j = H
                    elif new_alpha_unc_j <= L:
                        new_alpha_clippd_j = L
                    else:
                        new_alpha_clippd_j = new_alpha_unc_j

                    if(np.abs(old_alpha_j-new_alpha_clippd_j) <= 1e-5):
                        j += 1
                        continue
                    
                    break
                
                if j >= self.x_num:
                    continue
                    
                self.alpha[j] = new_alpha_clippd_j
                
                old_alpha_i = self.alpha[i]
                self.alpha[i] = old_alpha_i + self.Y_data[i] * self.Y_data[j] * (old_alpha_j - self.alpha[j])

                old_b = self.b

                bi_new = -self.E[i] - self.Y_data[i] * self.kernel_matrix[i][i] * (self.alpha[i] - old_alpha_i) \
                             - self.Y_data[j] * self.kernel_matrix[i][j] * (self.alpha[j] - old_alpha_j) + old_b

                bj_new = -self.E[j] - self.Y_data[i] * self.kernel_matrix[i][j] * (self.alpha[i] - old_alpha_i) \
                             - self.Y_data[j] * self.kernel_matrix[j][j] * (self.alpha[j] - old_alpha_j) + old_b
                
                if self.alpha[i] > 0 and self.alpha[i] < self.C:
                    self.b = bi_new
                elif self.alpha[j] > 0 and self.alpha[j] < self.C:
                    self.b = bj_new
                else:
                    self.b = (bi_new + bj_new)/2.0

                self.update_E(i)
                self.update_E(j)
                
                pair_changed += 1
                if pair_changed % 10 == 0:
                    self.valid()
        

    def init_kernel_matrix(self):
        print('start initing kernel matrix...')
        num = self.x_num
        for i in tqdm(range(0, num)):
            for j in range(0, num):
                if i > j:
                    self.kernel_matrix[i][j] = self.kernel_matrix[j][i]
                    continue
                if self.kernel_type == 'rbf':
                    self.kernel_matrix[i][j] = self.rbf_kernel(self.X_data[i], self.X_data[j])
                elif self.kernel_type == 'poly':
                    self.kernel_matrix[i][j] = self.polynominal_kernel(self.X_data[i], self.X_data[j])
        print('finish initing kernel matrix...')
    
    
    def init_valid_kernel_matrix(self):
        print('start initing valid kernel matrix...')
        for i in range(self.x_num):
            for j in range(len(self.valid_y)):
                if self.kernel_type == 'rbf':
                    self.valid_kernel_matrix[i][j] = self.rbf_kernel(self.X_data[i], self.valid_x[j])
                elif self.kernel_type == 'poly':
                    self.valid_kernel_matrix[i][j] = self.polynominal_kernel(self.X_data[i], self.valid_x[j])
        print('finish initing valid kernel matrix...')
        

    def update_E(self, index):
        
        sv = self.get_support_vectors()
        s = 0
        for i in range(len(sv)):
            s += self.Y_data[i]*self.alpha[i]*self.kernel_matrix[i][index]
        s += self.b - self.Y_data[index]
        
       
        self.E[index] = s
        
        

    def init_E(self):
        print('start initing E...')
        for i in range(self.x_num):
            self.E[i] = self.g_x_with_kernel_matrix(i) - self.Y_data[i]
        print('finish initing E...')
    

    def predict(self, x, opt):
        s = 0
        if opt == 'valid':
            for i in range(self.x_num):
                s += self.alpha[i] * self.Y_data[i] * self.valid_kernel_matrix[i][x]
            s += self.b
        elif opt == 'test':
            for i in range(self.x_num):
                if self.kernel_type == 'rbf':
                    s += self.best_alpha[i] * self.Y_data[i] * self.rbf_kernel(self.X_data[i], x)
                elif self.kernel_type == 'poly':
                    s += self.best_alpha[i] * self.Y_data[i] * self.polynominal_kernel(self.X_data[i], x)
            s += self.best_b
        
        return np.sign(s)
    

    def g_x(self, x):
        s = 0
        for k in range(self.x_num):
            if self.kernel_type == 'rbf':
                s += self.alpha[k]*self.Y_data[k]*self.rbf_kernel(x, self.X_data[k])
            elif self.kernel_type == 'poly':
                s += self.alpha[k]*self.Y_data[k]*self.polynominal_kernel(x, self.X_data[k])
        return s + self.b
    
    

    def g_x_with_kernel_matrix(self, i):
        s = 0
        for k in range(self.x_num):
            s += self.alpha[k]*self.Y_data[k]*self.kernel_matrix[i][k]
        
        return s + self.b
    
    
    def select_j(self, i):
        max_abs = 0
        max_index = -1
        for j in range(self.x_num):
            if i == j:
                continue
                
            temp_abs = np.abs(self.E[i]-self.E[j])
            if temp_abs > max_abs:
                max_abs = temp_abs
                max_index = j
        
        return max_abs, max_index
    
    
    def rbf_kernel(self, x, z):
        gamma = 0.5 # gamma > 0
        return np.exp(-gamma*np.linalg.norm(x-z))
    

    def get_support_vectors(self):
        sv = []
        for i in range(self.x_num):
            if self.alpha[i] > 0 and self.alpha[i] < self.C:
                sv.append(i)
                
        return sv
    
    
    def predict_of_probability(self, x):
        s = 0
       
        for i in range(self.x_num):
            if self.kernel_type == 'rbf':
                s += self.best_alpha[i] * self.Y_data[i] * self.rbf_kernel(self.X_data[i], x)
            elif self.kernel_type == 'poly':
                s += self.best_alpha[i] * self.Y_data[i] * self.polynominal_kernel(self.X_data[i], x)
                
        s += self.best_b
        
        return s
    
    
    def polynominal_kernel(self, x, z):
        return math.pow(np.dot(x, z) + 1, 3)
    
    
    def valid(self):
        y_predict = []
        for i in range(len(self.valid_x)):
            p = self.predict(i, opt='valid')
           
            y_predict.append(p)
        
        valid_f1 = f1_score(self.valid_y, y_predict)
        print('valid f1: {} precision: {} recall: {}'.format(valid_f1 , precision_score(self.valid_y, y_predict) \
                                                           , recall_score(self.valid_y, y_predict)))
        
        if valid_f1 >= self.best_f1:
            self.best_f1 = valid_f1
            self.best_alpha = copy.deepcopy(self.alpha)
            self.best_b = self.b
            print('           get new model')

In [4]:
def make_train_data_of_digit_i(digit):
    datas = []
    
    neg_sample_num = 0
    
    for i in range(len(X_train)):
        if y_train[i] != digit:
            datas.append([X_train[i], -1])
            neg_sample_num += 1
            if neg_sample_num >= 1000:
                break
                
                
    pos_sample_num = 0
    for i in range(len(X_train)):
        if y_train[i] == digit:
            datas.append([X_train[i], 1])
            pos_sample_num += 1
            
            if pos_sample_num >= 500:
                break
   
    shuffle(datas)
    data = []
    labels = []
    for i in range(len(datas)):
        data.append(datas[i][0])
        labels.append(datas[i][1])
    return data, labels


def make_valid_data_of_digit_i(digit):
    datas = []
    pos_sample_num = 0
    for i in range(len(X_valid)):
        if y_valid[i] == digit:
            datas.append([X_valid[i], 1])
            pos_sample_num += 1
            if pos_sample_num >= 200:
                break
    
    neg_sample_num = 0
    
    for i in range(len(X_valid)):
        if y_valid[i] != digit:
            datas.append([X_valid[i], -1])
            neg_sample_num += 1
            if neg_sample_num >= pos_sample_num:
                break
    
    shuffle(datas)
    data = []
    labels = []
    for i in range(len(datas)):
        data.append(datas[i][0])
        labels.append(datas[i][1])
        
    return data, labels

In [5]:
def train_single_model(C, kernel_type):
    svm = SVM(C=C)
    digit = 1
    
    train_data, train_labels = make_train_data_of_digit_i(digit)
    valid_data, valid_labels = make_valid_data_of_digit_i(digit)
    
    train_data = np.array(train_data)
    valid_data = np.array(valid_data)
    
    svm.train(train_data, train_labels, valid_data, valid_labels, epochs=1, kernel_type=kernel_type)
    
    _y_test = copy.deepcopy(y_test)
    
    for i in range(len(_y_test)):
        if _y_test[i] == digit:
            _y_test[i] = 1
        else:
            _y_test[i] = -1
        
    
    predict_list = []
    for i in tqdm(range(len(X_test[:1000]))):
        p = svm.predict(X_test[i], opt='test')
        predict_list.append(p)
        
    print('test f1: {} precision: {} recall: {}'.format(f1_score(_y_test[:1000], predict_list) , \
                                                            precision_score(_y_test[:1000], predict_list) \
                                                           , recall_score(_y_test[:1000], predict_list)))
# single_model(0.1, 'rbf')
train_single_model(0.001, 'poly')
# single_model(10, 'rbf')
# single_model(100, 'rbf')
# single_model(1000, 'rbf')
# single_model(10000, 'rbf')
# single_model(100000, 'rbf')

start initing valid kernel matrix...


  1%|▊                                                                              | 16/1500 [00:00<00:09, 157.28it/s]

finish initing valid kernel matrix...
start initing kernel matrix...


100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:06<00:00, 248.21it/s]


finish initing kernel matrix...
start initing E...
finish initing E...
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision

...

valid f1: 0.0 precision: 0.0 recall: 0.0
valid f1: 0.0676328502415459 precision: 1.0 recall: 0.035
valid f1: 0.0 precision: 0.0 recall: 0.0
valid f1: 0.8547008547008548 precision: 0.746268656716418 recall: 1.0
valid f1: 0.8565310492505354 precision: 0.7490636704119851 recall: 1.0


  2%|█▏                                                                             | 15/1000 [00:00<00:06, 144.62it/s]

valid f1: 0.8372093023255813 precision: 1.0 recall: 0.72


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 138.83it/s]


test f1: 0.9402390438247011 precision: 0.8939393939393939 recall: 0.9915966386554622


In [6]:
models = []
C_list = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]

for i in range(10):
    print('* train custom model for digit {}'.format(i))
    model = SVM(C_list[i])
    train_data, train_labels = make_train_data_of_digit_i(i)
    valid_data, valid_labels = make_valid_data_of_digit_i(i)
    
    train_data = np.array(train_data)
    valid_data = np.array(valid_data)
    
    model.train(train_data, train_labels, valid_data, valid_labels, epochs=1, kernel_type='poly')
    models.append(model)

* train custom model for digit 0
start initing valid kernel matrix...


  1%|▊                                                                              | 15/1500 [00:00<00:10, 146.02it/s]

finish initing valid kernel matrix...
start initing kernel matrix...


100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:06<00:00, 240.48it/s]


finish initing kernel matrix...
start initing E...
finish initing E...
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.8556962025316455 precision: 0.8666666666666667 recall: 0.845
           get new model
valid f1: 0.7055016181229774 precision: 1.0 recall: 0.545
valid f1: 0.9035532994923858 precision: 0.9175257731958762 recall: 0.89
           get new model
valid f1: 0.8338192419825073 precision: 1.0 recall: 0.715
valid f1: 0.8712328767123287 precision: 0.9636363636363636 recall: 0.795
valid f1: 0.8712328767123287 precision: 0.9636363636363636 recall: 0.795
valid f1: 0.8847184986595175 precision: 0.953757225433526 recall: 0.825
valid f1: 0.9104859335038362 precision: 0.9319371727748691 recall: 0.89
           get new model
valid f1: 0.7429643527204502 precision: 0.5945945945945946 recall: 0.99
valid f1: 0.888888888888889 precision: 0.9438202247191011 recall: 0.84
valid f1: 0.7987987987987988 precision: 1.0 recall: 0.665
valid f1: 0.769230769

...

finish initing kernel matrix...
start initing E...
finish initing E...
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6688963210702341 precision: 0.5025125628140703 recall: 1.0
           get new model
valid f1: 0.536231884057971 precision: 0.9736842105263158 recall: 0.37
valid f1: 0.5460992907801419 precision: 0.9390243902439024 recall: 0.385
valid f1: 0.6687898089171975 precision: 0.9210526315789473 recall: 0.525
valid f1: 0.5793103448275863 precision: 0.9333333333333333 recall: 0.42
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6711409395973155 precision: 0.5050505050505051 recall: 1.0
           get new model
valid f1: 0.7561436672967864 precision: 0.60790273556231 recall: 1.0
           get new model
valid f1: 0.6814310051107325 precision: 0.5167958

  1%|▉                                                                              | 17/1500 [00:00<00:09, 162.34it/s]

finish initing valid kernel matrix...
start initing kernel matrix...


100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:06<00:00, 218.90it/s]


finish initing kernel matrix...
start initing E...
finish initing E...
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6768189509306262 precision: 0.5115089514066496 recall: 1.0
           get new model
valid f1: 0.7155635062611807 precision: 0.5571030640668524 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6745362563237773 precision: 0.5089058524173028 recall: 1.0
valid f1: 0.6700167504187605 precision: 0.5037783375314862 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
valid f1: 0.6779661016949152 precision: 0.5128205128205128 recall: 1.0
valid f1: 0.6814310051107325 precision: 0.516795865

In [7]:
for model in models:
    print(model.best_f1)

0.9301204819277108
0.9774436090225564
0.9465648854961832
0.9211195928753181
0.9207920792079208
0.8726415094339623
0.942065491183879
0.9095477386934674
0.9207920792079208
0.9121951219512194


In [8]:
correct = 0
correct_list = []
error_list = []
for i in tqdm(range(len(X_test[:1000]))):
    prob_list = []
    for j in range(10):
        p = models[j].predict_of_probability(X_test[i])
        prob_list.append(p)
    
    p = np.argmax(prob_list)
    
    if p == y_test[i]:
        correct += 1
        correct_list.append(p)
    
    else:
        error_list.append([p, y_test[i]])

print('acc on test data of custom model: {}'.format(correct/len(X_test[:1000])))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:15<00:00, 13.25it/s]

acc on test data of custom model: 0.865


In [9]:
ans = []
for i in range(len(X_test[:1000])):
    ans.append(y_test[i])

a = dict(Counter(correct_list))
b = dict(Counter(ans))
for i in range(10):
    if i in a.keys():
        print('recall of digit {}: {}'.format(i, a[i]/b[i]))
    else:
        print('recall of digit {}: {}'.format(i, 0.0))

recall of digit 0: 0.898989898989899
recall of digit 1: 0.9915966386554622
recall of digit 2: 0.8514851485148515
recall of digit 3: 0.7816091954022989
recall of digit 4: 0.875
recall of digit 5: 0.7924528301886793
recall of digit 6: 0.73
recall of digit 7: 0.9313725490196079
recall of digit 8: 0.8554216867469879
recall of digit 9: 0.9090909090909091


In [10]:
s = []
for error in error_list:
    key = str(error[1]) + '->' + str(error[0])
    s.append(key)

s.sort()
Counter(s)

Counter({'0->2': 4,
         '0->4': 1,
         '0->5': 4,
         '0->6': 1,
         '1->6': 1,
         '2->0': 3,
         '2->1': 1,
         '2->3': 1,
         '2->4': 1,
         '2->5': 1,
         '2->7': 2,
         '2->8': 5,
         '2->9': 1,
         '3->0': 6,
         '3->1': 1,
         '3->2': 6,
         '3->5': 4,
         '3->7': 1,
         '3->8': 1,
         '4->1': 1,
         '4->5': 4,
         '4->6': 2,
         '4->8': 1,
         '4->9': 5,
         '5->0': 9,
         '5->2': 1,
         '5->3': 9,
         '5->7': 1,
         '5->8': 2,
         '6->0': 9,
         '6->2': 8,
         '6->4': 4,
         '6->5': 6,
         '7->2': 3,
         '7->5': 2,
         '7->8': 1,
         '7->9': 1,
         '8->0': 2,
         '8->2': 2,
         '8->3': 2,
         '8->5': 5,
         '8->9': 1,
         '9->0': 1,
         '9->1': 1,
         '9->5': 5,
         '9->6': 1,
         '9->7': 1})

In [11]:
sklearn_models = []
sklearn_C_list = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]

for i in range(10):
    print('* train sklearn model for digit {}'.format(i))
    model = SVC(C=sklearn_C_list[i], kernel='poly')
    train_data, train_labels = make_train_data_of_digit_i(i)
    valid_data, valid_labels = make_valid_data_of_digit_i(i)
    train_data += valid_data
    train_labels += valid_labels
    model.fit(train_data, train_labels)
    sklearn_models.append(model)

* train sklearn model for digit 0
* train sklearn model for digit 1
* train sklearn model for digit 2
* train sklearn model for digit 3
* train sklearn model for digit 4
* train sklearn model for digit 5
* train sklearn model for digit 6
* train sklearn model for digit 7
* train sklearn model for digit 8
* train sklearn model for digit 9


In [12]:
_correct = 0
for i in tqdm(range(len(X_test[:1000]))):
    prob_list = []
    for j in range(10):
        p = sklearn_models[j].decision_function(np.array(X_test[i]).reshape(1, -1))
        prob_list.append(p)
    
    p = np.argmax(prob_list)
    if y_test[i] == p:
        _correct += 1
    
print('acc on test data of sklearn model: {}'.format(_correct/len(X_test[:1000])))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 187.41it/s]

acc on test data of sklearn model: 0.868
